# 4.4 Implementing proofs using matrices and Ramsey’s Theorem

In [1]:
from z3 import *
import itertools

In [2]:
def product(A:list, B:list) -> list:
    n = len(A)
    res = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            
            minim = float('+inf')
            
            for k in range(n):
                aux = A[i][k]+B[k][j]
                if(aux < minim):
                    minim = aux
                    
            res[i][j] = minim
            
    return res            

In [3]:
def negative_diagonalQ(A:list) -> bool:
    i = 0
    n = len(A)
    found = False
    while i<n and not found:
        if A[i][i] < 0:
            found = True
        i += 1
    return found

In [4]:
def position_non_infinity(A:list) -> list:
    res = []
    for i in range(len(A)):
        for j in range(len(A[i])):
            if A[i][j] != float('inf'):
                res += [(i,j)]
    return res

In [5]:
def idempotentQ(A:list) -> bool:
    return position_non_infinity(A) == position_non_infinity(product(A, A))

In [6]:
def product_up2m_ngQ(matrix_list:list, m:int) -> bool:
    #check if any product of up to m matrixes doesn't satisfy negative_diagonalQ while being idempotent
    
    n = len(matrix_list)
    inp = [list(range(n))] * m
    product_index_list = list(itertools.product(*inp))

    len_tup = len(product_index_list)
    
    i = 0
    ok = True
    while i<len_tup and ok:
        tup = product_index_list[i]
        
        aux = matrix_list[tup[0]]
        for j in range(1, m):
            if idempotentQ(aux) and not negative_diagonalQ(aux):
                ok = False
                break
            aux = product(aux, matrix_list[tup[j]])
            
        if idempotentQ(aux) and not negative_diagonalQ(aux):
            ok = False
        
        i += 1
    
    return ok

In [7]:
def function2matrix(G:str) -> list:
    #given '(x1-1, x2)' construct the matrix [[-1, float('inf')],[float('inf'), 0]]
    
    #remove spaces from G
    g = ''
    for c in G:
        if c!=' ':
            g += c
    g = g[1:-1] #remove parentheses
    
    comma_index = [i for i in range(len(g)) if g[i]==',']
    gs = [g[0:comma_index[0]]] #projection 1 of g
    gs += [g[comma_index[i]+1:comma_index[i+1]] for i in range(len(comma_index)-1)]
    gs += [g[comma_index[-1]+1:]] #projection m of g
    
    dim = len(comma_index) + 1 #dimension of g
    
    n = Int('n')
    for i in range(dim):
        exec("x%d = Int('x%d')"%(i+1,i+1))
    
    matrix_result = [[0 for _ in range(dim)] for _ in range(dim)]
    
    for i in range(dim):
        for j in range(dim):
            try:
                #evaluates the difference and atributes if it is an integer
                aux = gs[j] + "-" + "x%s"%(i+1)
                matrix_result[i][j] = simplify(eval(aux)).as_long()
            except AttributeError:
                #if its not an integer set it to +oo
                matrix_result[i][j] = float('inf')
                
    return matrix_result
    

In [8]:
def main():
    #the output is 1 if the program terminates, 0 if it doesn't and 'unknown' if it can't decide.
    max_exp = int(input('maximum exponent = '))
    
    m = int(input('m = '))
    
    print('\nx = (input(Z), ..., input(Z))')
    print('while x > 0:')
    com = ''
    if m<6:
        for i in range(m):
            com += '%d,'%(i+1)
        com = com[:-1] #to exclude the last comma
    else:
        com = '1, 2, ..., %d, %d'%(m-1,m)

    print('   command = input(%s)'%com)
    
    matrix_list = []
    
    for i in range(1, m+1):
        print('   if command == %d:'%i)
        g = input('      x = ')
        matrix_list += [function2matrix(g)]
        
    ans = product_up2m_ngQ(matrix_list, max_exp)
    
    if ans:
        print("\nUp to exponent %s it didn't found problems."%max_exp)
        return 1
    
    else:
        print('unknown')
        return 'unknown' 

### Example - Program 5

In [9]:
main()

maximum exponent = 7
m = 3

x = (input(Z), ..., input(Z))
while x > 0:
   command = input(1,2,3)
   if command == 1:
      x = (x1-1, x2+n, x3, x4)
   if command == 2:
      x = (x1, x2-1, x3+n, x4)
   if command == 3:
      x = (x1, x2, x3-1, x4+n)

Up to exponent 7 it didn't found problems.


1